# Capítulo 2 - Pré-processamento dos dados

## Bibliotecas básicas e outras inicializações

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## Carregamento dos dados

In [2]:
dados = pd.read_csv('./datasets/mammographic_masses.data', 
                    names=['bi-rads', 'idade', 'forma_da_massa', 'contorno_da_massa', 'densidade_da_massa', 'severidade'],
                    usecols=['bi-rads', 'idade', 'forma_da_massa', 'contorno_da_massa', 'densidade_da_massa', 'severidade'],
                    na_values='?')
dados.head()

,bi-rads,idade,forma_da_massa,contorno_da_massa,densidade_da_massa,severidade
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


In [3]:
print(f'Linhas: {dados.shape[0]} | Colunas: {dados.shape[1]}')

Linhas: 961 | Colunas: 6


In [4]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961 entries, 0 to 960
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bi-rads             959 non-null    float64
 1   idade               956 non-null    float64
 2   forma_da_massa      930 non-null    float64
 3   contorno_da_massa   913 non-null    float64
 4   densidade_da_massa  885 non-null    float64
 5   severidade          961 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 45.2 KB


### Dicionário dos dados

**bi-rads**: sistema de clasificação adotado para estimar qual a chance de determinada imagem da mamografia ser câncer.{1-5}(float)

**idade**: idade do paciente em anos.(float)

**forma_da_massa**: {redonda=1, ovular=2, lobular=3, irregular=4}(float)

**contorno_da_massa**: {circunscrita=1, microlobulada=2, obscura=3, mal definida=4, especulada=5}(float)

**densidade_da_massa**: {alta=1, iso=2, baixa=3, gordurosa=4}(float)

**severidade**: {benigno=0, maligno=1}(inteiro)

## Limpeza da base

### Valores ausentes

#### Ignorar o objeto

In [5]:
dados_copy = dados.copy()
print(f'Antes: {dados_copy.shape}')
dados_copy.dropna(inplace=True)
print(f'Depois: {dados_copy.shape}')

Antes: (961, 6)
Depois: (830, 6)


In [6]:
print(f'Foram removidos {round((961-830)*100/961, 2)}% da base')

Foram removidos 13.63% da base


#### Imputar manualmente os valores ausentes

In [7]:
dados_copy = dados.copy()
print('Total de dados ausentes na coluna densidade_da_massa: ', dados_copy['densidade_da_massa'].isnull().sum())
dados_copy['densidade_da_massa'].fillna(1, inplace=True)
print('Total de dados ausentes na coluna densidade_da_massa: ', dados_copy['densidade_da_massa'].isnull().sum())

Total de dados ausentes na coluna densidade_da_massa:  76
Total de dados ausentes na coluna densidade_da_massa:  0


#### Imputar de acordo com a última observação (last observation carried forward)

In [8]:
dados_copy = dados.copy()
print('Total de dados ausentes na coluna contorno_da_massa: ', dados_copy['contorno_da_massa'].isnull().sum())
dados_copy['contorno_da_massa'].fillna(method='ffill', inplace=True)
print('Total de dados ausentes na coluna contorno_da_massa: ', dados_copy['contorno_da_massa'].isnull().sum())

Total de dados ausentes na coluna contorno_da_massa:  48
Total de dados ausentes na coluna contorno_da_massa:  0


#### Imputação do tipo hot-deck

In [9]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5, weights='distance')
dados_copy = dados.copy()

print('Total de dados ausentes na coluna idade: ', dados_copy['idade'].isnull().sum())
dados_copy['idade'] = imputer.fit_transform(dados_copy[['idade']])
print('Total de dados ausentes na coluna idade: ', dados_copy['idade'].isnull().sum())

Total de dados ausentes na coluna idade:  5
Total de dados ausentes na coluna idade:  0


#### Imputação com média, mediana ou moda

In [10]:
dados_copy = dados.copy()
dados_copy['idade'] = dados_copy['idade'].fillna(dados_copy['idade'].mean())
dados_copy['contorno_da_massa'] = dados_copy['contorno_da_massa'].fillna(dados_copy['contorno_da_massa'].median())

#### Imputação com média, mediana ou moda de objetos da mesma classe

In [11]:
dados_copy = dados.copy()
dados_copy['idade'] = dados_copy['idade'].fillna(dados_copy[dados_copy['severidade']==0]['idade'].mean())
dados_copy['idade'] = dados_copy['idade'].fillna(dados_copy[dados_copy['severidade']==1]['idade'].mean())

#### Usando modelos preditivos

In [12]:
from sklearn.tree import DecisionTreeClassifier

dados_copy = dados.copy()

test = dados_copy[dados_copy['idade'].isnull()]
test.drop(['idade'], 1, inplace=True)
dados_copy.dropna(inplace=True)

X = dados_copy.drop(['idade'], 1)
y = dados_copy['idade']


model = DecisionTreeClassifier()
model.fit(X, y)
nova_idade = model.predict(test)
test['idade'] = nova_idade

dados_copy = dados.copy()
dados_copy['idade'].fillna(value=test['idade'], inplace=True)

#### Tratando valores ausentes na base original

In [17]:
for col in ['bi-rads', 'idade', 'forma_da_massa', 'contorno_da_massa', 'densidade_da_massa']:
    dados[col].fillna(method='ffill', inplace=True)
    
print(f'Valores nulos na base: \n{dados.isnull().sum()}')

Valores nulos na base: 
bi-rads               0
idade                 0
forma_da_massa        0
contorno_da_massa     0
densidade_da_massa    0
severidade            0
dtype: int64


### Dados ruidosos